In [1]:
import os
import json
import glob as glob

In [2]:
from huggingface_hub import login

# non-blocking login
login(token="<HUGGINGFACETOKENHERE>")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /home/datalore/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset

data_files = {"train" : "result.json"}
my_data = load_dataset("json", data_files=data_files)

my_data_train_test = my_data["train"].train_test_split(test_size = 0.2)
my_data_train_test

Dataset json downloaded and prepared to /home/datalore/.cache/huggingface/datasets/json/default-91b230ce0a827a88/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['meeting_id', 'summary', 'dialogue'],
        num_rows: 135
    })
    test: Dataset({
        features: ['meeting_id', 'summary', 'dialogue'],
        num_rows: 34
    })
})

In [4]:
my_sample = my_data_train_test["train"].shuffle(seed=42).select(range(3))
for row in my_sample:
    print(f"\n'>>> Text: {row['dialogue']}'")


'>>> Text: SPEAKER_02: Excellent. So I sent you the agenda that was in the project document, I don't know if you've got a chance to just have a look at it.
SPEAKER_02: Anyway, the meeting is going to follow more or less the same structure as last time, so we'll go around each of you in turn and you can give your presentations on what you've been up to.
SPEAKER_02: And at the end of that we need to discuss what you've come up with so that we can make a decision on the key remote control concepts.
SPEAKER_02: So that's we need to know about the components, properties, materials, the user interface and any trends that the marketing expert has been watching.
SPEAKER_02: Okay, do you understand again? Okay. I've got 40 minutes.
SPEAKER_00: So I haven't played a PowerPoint. We have a bit of a passion.
SPEAKER_00: I thought I'll use the whiteboard instead.
SPEAKER_00: I'll put that back in and go down.
SPEAKER_00: Okay, so basically I'll start off by, I thought I'll use the whiteboard becaus

## Training Scripts

In [5]:
from datasets import concatenate_datasets
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorWithPadding
checkpoint = "philschmid/bart-large-cnn-samsum"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([my_data_train_test["train"], my_data_train_test["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([my_data_train_test["train"], my_data_train_test["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")



Max source length: 1024
Max target length: 201


In [7]:
# Use a pre-trained tokenizer to tokenize the dataset because model needs tensor input.

def preprocess_function(data, padding="max_length"):
    model_inputs = tokenizer(data["dialogue"], max_length=max_source_length, padding=padding, truncation=True)

    labels = tokenizer(text_target=data["summary"], max_length=max_target_length, padding=padding, truncation=True)
    
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs




In [8]:
tokenized_dataset = my_data_train_test.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "meeting_id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [9]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# Helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /home/datalore/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(checkpoint)
config.max_length = 400
config.min_length = 10
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, config = config)




In [12]:
# Verify max_length and min_length has been changed
model.generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 400,
  "min_length": 10,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

In [13]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [14]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

repository_id = f"{checkpoint.split('/')[1]}-icsi-ami-v2"
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=10,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

print("Done.")

Done.


In [15]:
import torch
torch.cuda.empty_cache()

In [16]:
trainer.train()

/opt/python/envs/default/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 135
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 1350
  Number of trainable parameters = 406290432
***** Running Evaluation *****
  Num examples = 34
  Batch size = 1
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 400,
  "min_length": 10,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.230755,38.427400,13.646100,22.366000,35.235300,185.794118
2,No log,3.302587,40.174800,11.794400,23.265500,36.471800,146.852941
3,No log,3.519942,39.820900,12.162100,22.777200,36.496700,141.764706
4,2.213100,4.050838,40.432500,11.654700,22.995800,36.878200,131.441176
5,2.213100,4.698793,38.409700,9.830900,20.389400,34.196700,145.970588
6,2.213100,4.959008,38.575800,9.633500,20.865000,35.032100,169.235294
7,2.213100,5.426377,38.281300,9.576400,21.140600,34.598900,148.029412
8,0.401000,5.488677,38.301400,9.688100,21.239800,34.158400,139.352941
9,0.401000,5.804439,39.960300,10.432900,22.689500,36.240600,145.235294
10,0.401000,5.898717,39.092800,10.840800,21.913800,35.506700,138.794118


TrainOutput(global_step=1350, training_loss=0.9856559583875868, metrics={'train_runtime': 2260.1119, 'train_samples_per_second': 0.597, 'train_steps_per_second': 0.597, 'total_flos': 2925591212851200.0, 'train_loss': 0.9856559583875868, 'epoch': 10.0})

In [24]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 51
  Batch size = 1
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 142,
  "min_length": 56,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 142,
  "min_length": 56,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty"

[51/51 01:58]

{'eval_loss': 3.318760633468628,
 'eval_rouge1': 37.6295,
 'eval_rouge2': 10.8682,
 'eval_rougeL': 21.9427,
 'eval_rougeLsum': 34.6403,
 'eval_gen_len': 127.62745098039215,
 'eval_runtime': 122.1386,
 'eval_samples_per_second': 0.418,
 'eval_steps_per_second': 0.418,
 'epoch': 5.0}

In [17]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()

tokenizer config file saved in bart-large-cnn-samsum-icsi-ami-v2/tokenizer_config.json
Special tokens file saved in bart-large-cnn-samsum-icsi-ami-v2/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 39.0928}]}


In [18]:
sample = """
None: Can you help me?
SPEAKER_02: No.
None: You okay?
SPEAKER_00: You okay?
SPEAKER_00: Okay?
SPEAKER_00: Is that okay?
None: Come on.
SPEAKER_00: Okay?
SPEAKER_00: Okay.
SPEAKER_00: All right, so start of the first meeting.
SPEAKER_00: All right, so agenda of the first meeting.
SPEAKER_00: Are we...
SPEAKER_00: We have 25 minutes for this meeting.
SPEAKER_00: We...
SPEAKER_00: Are to get acquainted.
SPEAKER_00: So, does everyone want to say who they are?
SPEAKER_01: And that's sensible.
SPEAKER_02: I'm Robin. I'm a marketing manager.
SPEAKER_00: I'm Lisa. I'm a music designer.
SPEAKER_00: I'm Nick. I'm the industrial designer.
SPEAKER_00: And I'm Amostar. I'm on the project leader.
SPEAKER_00: Project plan.
SPEAKER_00: So, does anyone have any thoughts?
SPEAKER_00: As to the tool training that is required.
SPEAKER_02: I'm nice. I'll show you what you mean by tool training.
SPEAKER_00: Neither am I.
SPEAKER_00: Oh, I see. So, we shouldn't really be...
SPEAKER_00: All right. Okay.
SPEAKER_00: So, we have the project team, which is to basically to come up with a new remote control device.
SPEAKER_00: We have...
SPEAKER_00: The starting base was the original.
SPEAKER_00: Which has been an existence now for a period of time.
SPEAKER_00: And our idea is to make the new remote control device more user-friendly than the previous one.
SPEAKER_00: And to be trendy or to be with it, and therefore to get a bigger market share and bigger audience.
SPEAKER_00: So, the method of doing this is split up.
SPEAKER_00: As you can see into the functional design, the conceptual design, and the detailed design.
SPEAKER_00: So, in each of these phases, we'll basically be handing over to yourselves the designers of this...
SPEAKER_00: This device.
SPEAKER_00: And having meetings so that we can, during the course of the day, come up with a better implement than we had before.
SPEAKER_00: And therefore have a successful conclusion to the D.
SPEAKER_00: And you'll be doing various designs throughout the day to meet this end.
SPEAKER_00: So, we've got two chain.
SPEAKER_00: Try out whiteboard.
SPEAKER_00: So, we'll...
SPEAKER_00: Right. So, basically everyone's to supposedly draw their favourite animal over on the whiteboard over there.
SPEAKER_00: I guess this is...
SPEAKER_00: Make sure the whiteboard works.
SPEAKER_00: I don't know who wishes to go first.
SPEAKER_00: You wish to go...
SPEAKER_00: Have a first bash at whatever.
SPEAKER_02: Um...
SPEAKER_00: Uh...
SPEAKER_00: Good. I've got pocket study.
SPEAKER_00: But now you move out from the microphone, and the camera...
SPEAKER_00: What about you? That's right now, do you think?
SPEAKER_00: I would guess so.
SPEAKER_03: You've asked the microphone.
SPEAKER_00: Technical problems. I mean, you designers are meant to come up with it.
SPEAKER_00: So...
SPEAKER_00: Oh, no.
SPEAKER_02: Okay.
SPEAKER_02: I think that I would have to see that my favourite animal is the cat.
SPEAKER_02: What's my look at there?
SPEAKER_02: Um...
SPEAKER_02: And this would be because the very independent, the very intelligent, the compote the dogs maybe.
SPEAKER_02: And it can be very, very affectionate. Some people don't think so, but are not very affectionate cats.
SPEAKER_02: Um...
SPEAKER_02: And they come back to themselves.
None: Right.
SPEAKER_00: Next.
SPEAKER_02: Shall I look like that? Oh, actually.
SPEAKER_00: I don't see. There's any need to. There's plenty of space.
SPEAKER_00: I mean...
SPEAKER_01: Whatever. I'm a hominatry.
SPEAKER_03: Exactly.
SPEAKER_03: So I can get the casserole at this time.
SPEAKER_00: We've had more time to prepare over this site, so we've all stuck our bits in pieces in the pockets.
SPEAKER_03: I didn't think that.
SPEAKER_03: Okay.
SPEAKER_03: Three pencered.
SPEAKER_03: I'll try the red pen.
SPEAKER_03: Okay.
SPEAKER_03: Um...
SPEAKER_03: I'm gonna go for the bath.
SPEAKER_03: Wait, something else is draw very well.
SPEAKER_03: I'll have a bath.
SPEAKER_00: You get marks for artistic impression.
SPEAKER_03: Oh, I lost it there.
SPEAKER_03: I have to look something like a...
SPEAKER_00: Um... So you're just doing the face.
SPEAKER_00: I'll go for a...
SPEAKER_03: A small, small, bare button.
SPEAKER_03: Um...
SPEAKER_03: And I like my animal that looks nothing like a bath because, um...
SPEAKER_03: Has no, maybe because there's so many cartoon characters who've made a bath.
SPEAKER_01: Right.
SPEAKER_01: Hello. Um...
SPEAKER_01: I'm gonna go for the dog.
SPEAKER_01: I'm gonna draw one badly as well.
SPEAKER_01: Uh...
SPEAKER_01: Looks like he's going to get ducks and does something.
SPEAKER_02: That's very good.
SPEAKER_02: Right.
SPEAKER_01: There's my dog.
SPEAKER_01: Um... I like dogs because they're very loyal and they're always happy.
SPEAKER_01: So whenever you're feeling sort of a bit...
SPEAKER_01: Down or tired, they're always coming up and they're always, um, quite excited.
SPEAKER_01: So, um...
SPEAKER_01: You can always have a lot of fun with the dog and they're also good for exercises for you so they get out and they never get tired.
SPEAKER_01: And when they're tired, they're quite cute as well.
SPEAKER_01: Okay.
SPEAKER_01: That's why I like dogs.
SPEAKER_00: Alright.
SPEAKER_00: Well, I've not actually had too many pets over my time because, to be honest with you, uh...
SPEAKER_00: Not too keen on them.
SPEAKER_00: Anyway, not to worry.
SPEAKER_00: So what my daughters have got at the moment is they've got a few fish.
SPEAKER_00: And so hopefully, um, won't prove too difficult to draw.
SPEAKER_00: Uh...
None: Mm-hmm.
None: Mm-hmm.
SPEAKER_00: As you can see, my artistic work is useless as well.
SPEAKER_00: Anyway, um...
SPEAKER_00: One of the best things about fish is that they don't really take, uh, too much looking after because, uh, with most of the animals, if you're going away in holiday or whatever, you've got to spend money or get a friend or whatever to look after them for you.
SPEAKER_00: Whereas if you've got fish, you just gotta put the food in a dripper feed, which feeds them over a couple of weeks at your way and, uh...
SPEAKER_00: Change the water every couple of months and buying a few plants, so other than the fact they keep dying, uh, fish are not... are reasonable pets in that, uh, they're low maintenance.
SPEAKER_00: Right.
SPEAKER_00: Okay.
SPEAKER_00: Or still, what's...
SPEAKER_00: Right, okay.
SPEAKER_00: So, work has been done on, uh...
SPEAKER_00: This project whereby, um...
SPEAKER_00: 25 euros is the expected, uh...
SPEAKER_00: Sound price?
SPEAKER_00: Information has come from our marketing manager here.
SPEAKER_00: Mm-hmm.
SPEAKER_00: So, we're looking to sell internationally, not just in Europe.
SPEAKER_00: We're looking at, um...
SPEAKER_00: Having a production costs limited to, uh, 12.5 euro per unit.
SPEAKER_00: And therefore making a profit margin of, uh, well, not actually a profit margin, it's, uh...
SPEAKER_00: Because obviously you're going to have overheads in your southern costs to, uh, take, uh, from, uh...
SPEAKER_00: From that to give you your profit margin per unit.
SPEAKER_00: And so depending on what the, uh, the overhead costs are, will determine, uh...
SPEAKER_00: How many units were, uh, looking to sell, or projecting to sell at this point in time.
SPEAKER_00: So, um...
SPEAKER_00: Experience with remote control, first idea is...
SPEAKER_00: You're remote. So, I guess we're looking at, um...
SPEAKER_00: I'm going to be discussing at this point in time to help you, um...
SPEAKER_00: Folks design our new model, as it were. So, uh...
SPEAKER_00: Any, any thoughts?
SPEAKER_03: Um...
SPEAKER_03: I maybe found myself about controls a bit, but it's a little small.
SPEAKER_03: But I had to pass. So we could make something with a...
SPEAKER_03: This is passed, but...
SPEAKER_03: As I was making a function.
SPEAKER_00: Okay, so...
SPEAKER_00: So basically we're looking for some, um...
SPEAKER_00: We're looking for a device that is, um...
SPEAKER_00: Robost.
SPEAKER_00: And...
SPEAKER_00: And therefore, uh...
SPEAKER_00: I'm going to get damaged too easily.
SPEAKER_00: Um... I'm looking for a device.
SPEAKER_00: That is, uh...
SPEAKER_00: What was the other thing you just said there?
SPEAKER_03: Um... So, easy to use.
SPEAKER_03: Easy to use.
SPEAKER_03: That's what I was able to say.
SPEAKER_03: Yeah.
SPEAKER_00: Yes.
SPEAKER_03: This is easy to use and see.
SPEAKER_00: And see.
SPEAKER_00: Okay.
SPEAKER_00: Uh...
SPEAKER_00: Could we just check?
SPEAKER_02: Is this just a television?
SPEAKER_02: Because a lot of them...
SPEAKER_02: Systems are kind of TV video combined now. TV, TV, TV combined.
SPEAKER_02: And one of the most annoying things is having like five and more to the house.
SPEAKER_02: So if you've got a combined system, it could be a combined law.
SPEAKER_02: Or is it just a television that's supposed to do?
SPEAKER_00: Um...
SPEAKER_00: They still get back to you on that, but it seems to me sensible because, as you rightly said, there's nothing more annoying than having three or four devices littered about the, uh...
SPEAKER_00: About the room and, uh...
SPEAKER_00: So, a device for...
SPEAKER_00: For all remotes.
SPEAKER_03: So, it's a little...
SPEAKER_01: Okay. Yeah.
SPEAKER_01: Um... One of the things we found in the market research is that people have to get confused by the number of buttons on them as well.
SPEAKER_01: Because there's quite often lots and lots.
SPEAKER_01: And, um...
SPEAKER_01: Sometimes, uh...
SPEAKER_01: They'd sort of remote controls to be their own purpose because you're sat in the chair and the remote is somewhere else in the room.
SPEAKER_01: So, whereas in the past you don't have to get up to change the channel.
SPEAKER_01: Now you have to get up to sort of pick up the remote.
SPEAKER_01: So, I don't...
SPEAKER_01: We need to sort of maybe think about how...
SPEAKER_01: We could maybe, uh...
SPEAKER_01: The remote control which moves around the room.
SPEAKER_01: That's maybe something for the future when you talk to your television.
SPEAKER_00: That is, in a sense, it's mutually exclusive.
SPEAKER_00: You can't have both the one device and then have a few buttons on it too.
SPEAKER_00: Because you want simplicity as well.
SPEAKER_00: You want any idiot to be able to use it.
SPEAKER_00: Whilst at the same time you want, as you rightly say, one remote for all.
SPEAKER_00: And so, these are probably mutually exclusive options.
SPEAKER_00: That, uh...
SPEAKER_00: You could argue that experience of using devices and similar devices as people get more and more used to using remotes.
SPEAKER_00: Therefore, they're more often with handling them.
SPEAKER_00: Therefore, you can make them more complicated as time goes on.
SPEAKER_03: They would have better instructions with the remote.
SPEAKER_03: Or we just do the design of the remote control itself or the instructions that we can with it.
SPEAKER_00: There's instructions.
SPEAKER_01: Yeah.
SPEAKER_01: I mean, we've done some research about sort of, you know, what the cutting edge sort of handheld devices are.
SPEAKER_01: And a lot of them sort of use, you know, they're like mini laptops.
SPEAKER_01: So it's possible that we could devise a system where you're basically sort of holding a miniature computer which is controlling all your sort of your television, your stereo.
SPEAKER_01: And where, you know, if you buy a new thing, then it's sort of, you can link it to that as well.
SPEAKER_00: Okay.
SPEAKER_00: Well, we've got five minutes before the end of the meeting.
SPEAKER_00: So, we have to start winding up.
SPEAKER_00: Is there next meeting in 30 minutes?
SPEAKER_00: Okay.
SPEAKER_00: So, um, right.
SPEAKER_00: So we've got ID, the command of mine.
SPEAKER_01: You just click return.
SPEAKER_01: Okay.
SPEAKER_01: Get rid of the message.
SPEAKER_03: Or not.
SPEAKER_01: If you hit just hit return and it should get rid of the message.
SPEAKER_00: Oh, there we go.
SPEAKER_00: Yeah.
SPEAKER_00: That's what I was looking for.
SPEAKER_00: Right.
SPEAKER_00: So we've got functional.
SPEAKER_02: What happened to the.
SPEAKER_02: Yeah.
SPEAKER_00: Sorry about that.
SPEAKER_00: Okay.
SPEAKER_00: So, um, the working design for ID, the, the technical functions design, marketing, the user requirement specification specific instructions sent to you about your person, by your personal coach.
SPEAKER_00: So we all clear what objectives we're looking to meet in the next 30 minutes.
SPEAKER_00: And I guess I'll try and write up some minutes of this meeting to, uh, to give it to you for the next meeting.
SPEAKER_02: I'm not necessarily clear on what we're designing.
SPEAKER_02: Is it a multi functional one or do we decide that ourselves?
SPEAKER_02: So the more we work on it.
SPEAKER_03: I think you just said the start of the television about television.
SPEAKER_00: That's true.
SPEAKER_00: During the course of our day, we might make decisions based on information or meetings that would change.
SPEAKER_00: Okay.
SPEAKER_00: Where we're going.
SPEAKER_00: But at this point in time, I think you're right.
SPEAKER_00: So make it just a TV.
SPEAKER_00: Okay.
SPEAKER_00: So.
SPEAKER_00: Depart.
SPEAKER_00: We will stay here and, uh, and break off.
SPEAKER_00: So we'll see you in half now.
SPEAKER_00: Okay.
SPEAKER_00: Okay.
SPEAKER_00: Right.
None: Okay.
"""

In [20]:
from transformers import pipeline
summarizer = pipeline("summarization", model="./bart-large-cnn-samsum-icsi-ami-v2/checkpoint-135", tokenizer = checkpoint, truncation=True)


loading configuration file ./bart-large-cnn-samsum-icsi-ami-v2/checkpoint-135/config.json
Model config BartConfig {
  "_name_or_path": "./bart-large-cnn-samsum-icsi-ami-v2/checkpoint-135",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
  

In [59]:
res = summarizer(sample , max_length = 200)
print(f"{res[0]}")

{'summary_text': 'This is the first meeting of a new remote control project. Project Manager introduced the team to each other and Project Manager asked them to draw their favourite animals. The team agreed that the remote control should be more user-friendly than the previous one. The meeting ended with a general discussion on the design of the remote, including the functional design, conceptual design, and detailed design.'}


Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 142,
  "min_length": 56,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}



In [14]:
from evaluate import evaluator
import evaluate
metric = evaluate.load("rouge")
print(metric.inputs_description)



Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Returns:
    rouge1: rouge_1 (f1),
    rouge2: rouge_2 (f1),
    rougeL: rouge_l (f1),
    rougeLsum: rouge_lsum (f1)
Examples:

    >>> rouge = evaluate

In [56]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 34
  Batch size = 1
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 400,
  "min_length": 10,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty": 2.0,
  "max_length": 400,
  "min_length": 10,
  "no_repeat_ngram_size": 3,
  "num_beams": 4,
  "pad_token_id": 1,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "length_penalty"

[34/34 01:18]

{'eval_loss': 3.1403207778930664,
 'eval_rouge1': 38.7497,
 'eval_rouge2': 10.8123,
 'eval_rougeL': 22.6838,
 'eval_rougeLsum': 34.7075,
 'eval_gen_len': 138.11764705882354,
 'eval_runtime': 81.8894,
 'eval_samples_per_second': 0.415,
 'eval_steps_per_second': 0.415,
 'epoch': 5.0}

In [15]:
task_evaluator = evaluator("summarization")
eval_results = task_evaluator.compute(model_or_pipeline=summarizer
                                      , data = my_data_train_test["test"], input_column="dialogue", label_column="summary")
print(eval_results)

{'rouge1': 0.3904652959716078, 'rouge2': 0.14393620735702914, 'rougeL': 0.24746191692763833, 'rougeLsum': 0.2476617812520101, 'total_time_in_seconds': 526.7507517189997, 'samples_per_second': 0.06454665681832311, 'latency_in_seconds': 15.492669168205875}


In [16]:
old_summarizer = pipeline("summarization", model="vmarklynn/bart-large-cnn-samsum-acsi-ami", tokenizer = checkpoint, truncation = True)
eval_results_old = task_evaluator.compute(model_or_pipeline=old_summarizer
                                      , data = my_data_train_test["test"], input_column="dialogue", label_column="summary")
print(eval_results_old)

{'rouge1': 0.3686045117283241, 'rouge2': 0.12595355808827308, 'rougeL': 0.23618191347875278, 'rougeLsum': 0.23597138924820907, 'total_time_in_seconds': 527.3878147569994, 'samples_per_second': 0.06446868708118697, 'latency_in_seconds': 15.511406316382338}


In [17]:
train_eval_results = task_evaluator.compute(model_or_pipeline=summarizer
                                      , data = my_data_train_test["train"], input_column="dialogue", label_column="summary")
print(train_eval_results)

{'rouge1': 0.395087405543628, 'rouge2': 0.1342562863153256, 'rougeL': 0.24645589644585283, 'rougeLsum': 0.2461069082339275, 'total_time_in_seconds': 1996.8952170309994, 'samples_per_second': 0.06760494934767741, 'latency_in_seconds': 14.791816422451847}


In [ ]:
trainer.evaluate()